In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/0_isw_data_collection/1_isw_preprocessed.csv')

### NLTK

In [3]:
#!pip3 install --upgrade pip

In [4]:
#pip install num2words

In [5]:
!pip install scikit-learn

In [6]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words


import nltk
import os
import string
import numpy as np
import copy 
import pickle
import re
import math

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [7]:
data.head()

,Unnamed: 0,date,short_url,title,text_title,full_url,main_html,main_html_v6
0,0,2022-02-25,RusCampaignFeb25,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step..."
1,1,2022-02-26,RusCampaignFeb26,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Katya Stepane..."
2,2,2022-02-27,RusCampaignFeb27,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step..."
3,3,2022-02-28,RusCampaignFeb28,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step..."
4,4,2022-03-01,RusCampaignMar1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","Frederick W. Kagan, George Barros, and Katery..."


### Prepare functions

In [ ]:
def remove_one_letter_word(data):
    words = word_tokenize(str(data))
    
    new_text = ""
    for w in words:
        if (len(w) > 1):
            new_text = new_text + " " + w
            
    return new_text

In [ ]:
#remove_one_letter_word(data['main_html_v6'][0])

In [ ]:
def convert_lower_case(data):
    return np.char.lower(data)

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:
def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_stop_words = {"no","not"}
    
    stop_words = stop_words - stop_stop_words
    
    words = word_tokenize(str(data))
    
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
            
    return new_text

In [ ]:
#remove_stop_words(data['main_html_v6'][0])

In [ ]:
def remove_punctuations(data):
    symbols = " !()-[]{};:'\,<>./?@#$%^&*_~ '"""  
    
    for i in range(len(symbols)):
        data = np.char.replace(data,symbols[i]," ")
        data = np.char.replace(data,"  "," ")
    
    data = np.char.replace(data,",", " ")
    
    return data
            

In [ ]:
#remove_punctuations(data['main_html_v6'][0])

In [ ]:
def stemming(data):
    stemmer = PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
        
    return new_text

def lemmatizing(data):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
               
    return new_text

In [ ]:
def convert_numbers(data):
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        if w.isdigit():
            if int(w) < 1000000000:
                w = num2words(w)
            else:
                w = " "
        new_text = new_text + " " + w
            
    new_text = np.char.replace(new_text,"-"," ")
        
    return new_text
                
                

In [ ]:
#convert_numbers(data['main_html_v6'][0])

In [ ]:
def remove_url_from_string(data):
    words = word_tokenize(str(data))
    new_text = ""

    
    for w in words:
        w = re.sub(r'https?:\/\/.*[\r\n]*',"",str(w),flags=re.MULTILINE)
        w = re.sub(r'http?:\/\/.*[\r\n]*',"",str(w),flags=re.MULTILINE)
        
        new_text = new_text + " " + w
        
    return new_text

In [ ]:
def preprocess(data, word_root_algo='lemm'):
    
    data = remove_one_letter_word(data)
    data = convert_lower_case(data)
    data = remove_stop_words(data)
    data = stemming(data)
    data = remove_punctuations(data)
    data = convert_numbers(data)
    data = remove_url_from_string(data)
    
    if word_root_algo =='lemm':
        data = lemmatizing(data)
    else: 
        data = stemming(data)
        
    
    data = remove_punctuations(data)
    data = remove_stop_words(data)
    
    return data

In [ ]:
data['report_text_lemm'] = data['main_html_v6'].apply(lambda x: preprocess(x,"lemm"))

In [ ]:
data['report_text_stemm'] = data['main_html_v6'].apply(lambda x: preprocess(x,"stemm"))

In [ ]:
data.head()

In [ ]:
data['report_text_lemm'][0]

In [ ]:
docs = data['report_text_lemm'].tolist()

In [ ]:
data.to_csv('data/0_isw_data_collection/2_isw_preprocessed_before_vectosing.csv')

## CountVerctorizer

In [ ]:
cv = CountVectorizer(max_df=0.98,min_df=2)
word_count_vector = cv.fit_transform(docs)

word_count_vector.shape

In [ ]:
word_count_vector

In [ ]:
with open("models/count_vectorizer_v1.pkl","wb") as handle:
    pickle.dump(cv, handle)

### TF-IDF

In [ ]:
tfidf_transformer = TfidfTransformer(smooth_idf = True, use_idf = True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
with open("models/tfidf_transformer_v1.pkl","wb") as handle:
    pickle.dump(tfidf_transformer, handle)

In [ ]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index = cv.get_feature_names_out(),columns=["idf_weights"])
df_idf.sort_values(by=['idf_weights'])

In [ ]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

In [ ]:
tf_idf_vector

In [ ]:
tf_idf_vector.shape

### Transformation with fitted models

In [ ]:
tfidf = pickle.load(open("models/tfidf_transformer_v1.pkl","rb"))
cv = pickle.load(open("models/count_vectorizer_v1.pkl","rb"))

In [ ]:
feature_names = cv.get_feature_names_out()

In [ ]:
feature_names

In [ ]:
def sort_coo(coo_matrix):
    
    tuples = zip(coo_matrix.col, coo_matrix.data)
    
    return sorted(tuples,key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    
    sorted_items = sorted_items[:topn]
    
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        
        score_vals.append(round(score,3))
        feature_vals.append(feature_names[idx])
        
    results = {}
    
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]
    
    return results


def convert_doc_to_vector(doc):
    feature_names = cv.get_feature_names_out()
    top_n = 10
    tf_idf_vector = tfidf.transform(cv.transform([doc]))
    
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    
    keywords = extract_topn_from_vector(feature_names, sorted_items, top_n)
    
    return keywords

In [ ]:
data['keywords'] = data['report_text_stemm'].apply(lambda x: convert_doc_to_vector(x))

In [ ]:
data['keywords'][0]

In [ ]:
data_vectorised = tf_idf_vector.toarray()

In [ ]:
vectors_df = pd.DataFrame(data_vectorised)

In [ ]:
vectors_df.to_csv('data/0_isw_data_collection/3_isw_vectorised.csv')